In [21]:
# load the daily data, then calculate the value of it 

#df['value_ema_1_beta_99'] = df['diff_ema_1'].shift(-1).fillna(0) + \
#0.99 * df['value_ema_1_beta_99'].shift(-1).fillna(0)

In [22]:
import pandas as pd
import numpy as np
import os

In [23]:
def get_stock_value_df(stock_name):
    df = pd.read_json(f'../data/stocks/{stock_name}/daily/data')
    df['date']=pd.to_datetime(df['timestamp']).apply(lambda x: x.date())
    df['time']=pd.to_datetime(df['timestamp'])
    df['close_log'] = np.log(df['close'])
    df['close_log_diff']=df['close_log'].diff().fillna(0)
    df['value']=0
    for i in range(20):
        df['value']=df['close_log_diff'].shift(-1).fillna(0)+0.9*df['value'].shift(-1).fillna(0)
    return df[['time', 'value']]
    

In [49]:
def gradeToNum(grade):
    gradeDict = {
        'Buy':1,
        'Overweight':2,
        'Outperform':2,
        'Sell':5,
        'Neutral':3,
        'Hold':3,
        'Equal-Weight':3,
        'Underweight':4,
        'Market Outperform':2,
        'Market Perform':3,
        'Strong Buy': 0,
        'Peer Perform': 3,
        'Underperform':4,
        
        'Reduce':4,
        'Sector Weight':3,
        'Negative':4,
        'Perform':3,
        'Positive':2,
        'Sector Perform':3,
        'Equal-weight':3,
        'Long-Term Buy':2,
        'Long-term Buy':2,
        'Sector Outperform':2,
        'Average': 3,
        'In-Line': 2,
        'Market Underperform':4,
        'Top Pick': 1,
        'Sector Underperform': 4,
        '':3
    }
    if grade not in list(gradeDict.keys()):
        print(f'{grade} is not found in the dictionary')
        return 3
    return gradeDict[grade]

In [50]:
from pathlib import Path
def get_expert_votes(stock_name):
    vote_path = f'../data/stocks/{stock_name}/updown'
    try:
        df = pd.read_json(vote_path)
    except:
        print('reading json failed')
        return
    df['toGradeNum']=df['toGrade'].apply(gradeToNum)
    df['date']=pd.to_datetime(df['epochGradeDate'].astype('datetime64[s]')).apply(lambda x: x.date())
    df['time']=pd.to_datetime(df['epochGradeDate'].astype('datetime64[s]'))
    wide_table = df.pivot_table(index='time', columns='firm', values='toGradeNum').fillna(method='ffill').fillna(3)
    return wide_table


In [51]:
try:
  df = pd.read_json(f'../data/stocks/AMBK/updown')
except ValueError:
  print('format error')

format error


In [52]:
def get_final_df(stock_name):
    df1 = get_stock_value_df(stock_name)
    df2 = get_expert_votes(stock_name)
    if df2 is None:

        return None, None
    df_merged = pd.merge_asof(df1, df2.sort_values(by=['time']), on='time')
    # remove if all value is null
    df_merged = df_merged[df_merged[df2.columns].notnull().any(axis=1)]
    return df_merged, df2.columns

In [53]:
def get_corr(stock_name):
    final_df, firms = get_final_df(stock_name)
    if final_df is None:
        return
    corr_df = final_df[[*firms, 'value']].corr()['value'].reset_index().rename({'index': 'firm', 'value': stock_name}, axis=1)
    return corr_df


In [69]:
rootdir = '../data/stocks'
stock_list = os.listdir(rootdir)
df = None
for idx, stock_name in enumerate(stock_list):
    print(f'handling {stock_name}')
    if df is None:
        df = get_corr(stock_name)
    else:
        df_tmp = get_corr(stock_name)
        if df_tmp is not None:
            print(f'merging {stock_name}')
            df = pd.merge(df, df_tmp, on='firm', how='outer')
            print(df.head())

    

handling AMBK
reading json failed
handling HEXA-B.ST
reading json failed
handling PFE
handling AIR.F
reading json failed
handling RHHBY
merging RHHBY
                firm       PFE     RHHBY
0              Argus -0.035065 -0.002996
1  Atlantic Equities  0.099644       NaN
2  B of A Securities  0.039994       NaN
3        BMO Capital  0.026829       NaN
4    Bank of America  0.046797       NaN
handling NVDA
merging NVDA
                firm       PFE     RHHBY      NVDA
0              Argus -0.035065 -0.002996  0.066300
1  Atlantic Equities  0.099644       NaN -0.023069
2  B of A Securities  0.039994       NaN -0.007195
3        BMO Capital  0.026829       NaN  0.073811
4    Bank of America  0.046797       NaN -0.090834
handling MRK
merging MRK
                firm       PFE     RHHBY      NVDA       MRK
0              Argus -0.035065 -0.002996  0.066300       NaN
1  Atlantic Equities  0.099644       NaN -0.023069       NaN
2  B of A Securities  0.039994       NaN -0.007195       NaN
3 

In [71]:
df.fillna(0)

,firm,PFE,RHHBY,NVDA,MRK,MSFT,NSRGY,GOOGL,INTC,AMZN,DIS,TSLA,DDAIF,T,AAPL,TSM
0,Argus,-0.035065,-0.002996,0.066300,0.000000,-0.040123,0.0,0.013654,0.042025,0.103223,0.097371,0.045836,0.0,0.025980,-0.048681,0.000000
1,Atlantic Equities,0.099644,0.000000,-0.023069,0.000000,-0.006325,0.0,0.000000,-0.064691,0.000000,-0.006268,0.000000,0.0,0.000000,0.024181,0.233924
2,B of A Securities,0.039994,0.000000,-0.007195,0.000000,0.060750,0.0,-0.035469,0.061787,0.089476,0.151825,0.056846,0.0,0.015724,0.005313,0.000000
3,BMO Capital,0.026829,0.000000,0.073811,-0.032987,-0.069200,0.0,-0.039922,0.006871,0.022901,-0.062142,0.000000,0.0,0.000000,0.091797,0.000000
4,Bank of America,0.046797,0.000000,-0.090834,0.040338,-0.056026,0.0,0.040187,0.047537,0.060391,0.064124,0.210462,0.0,0.010279,-0.049526,0.053939
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,Instinet,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
153,Morgan Keegan,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.131954,0.000000
154,OTR Global,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.078421,0.000000
155,Oracle Investment Research,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.023782,0.000000


In [ ]:
import seaborn as sns

sns.relplot(x="horsepower", y="mpg", hue="origin", size="weight",
            sizes=(40, 400), alpha=.5, palette="muted",
            height=6, data=mpg)

In [62]:
pd.merge(get_corr('AAPL'), get_corr('NSRGY'), on='firm')

,firm,T,AAPL
0,Argus,0.025980,-0.048681
1,Argus Research,NaN,0.033995
2,B of A Securities,0.015724,0.005313
3,BMO Capital,NaN,0.091797
4,Baird,0.088960,-0.073927
5,Bank of America,0.010279,-0.049526
6,Bank oferica,NaN,0.071556
7,Barclays,-0.048874,0.135910
8,Bernstein,NaN,0.027304
9,Canaccord Genuity,NaN,-0.138179


In [56]:
df

,firm,PFE,RHHBY,NVDA,MRK,MSFT,NSRGY,GOOGL,INTC,AMZN,DIS,TSLA,DDAIF,T,AAPL,TSM
0,value,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [143]:
rootdir = '../data/stocks'
stock_list = os.listdir(rootdir)

#   for dir in dirs:
#     name = dir
#     if '=' in name:
#       indicators_list.append(name)

In [174]:
import math

test = pd.DataFrame({'A': [1,2, math.nan], 'B': [0,math.nan,math.nan]})

In [176]:
test.notnull().any(axis=1)

0     True
1     True
2    False
dtype: bool

In [165]:
test

,A,B
0,1.0,0.0
1,2.0,1.0
2,NaN,NaN


In [139]:
get_stock_value_df('AAPL')

,timestamp,open,high,low,close,volumn,date,time,close_log,close_log_diff,AAPL_value
0,2002-07-09 13:30:00,0.323036,0.326607,0.311786,0.313036,226749600,2002-07-09,2002-07-09 13:30:00,-1.161437,0.000000,-0.090168
1,2002-07-10 13:30:00,0.316250,0.324464,0.308036,0.309286,206880800,2002-07-10,2002-07-10 13:30:00,-1.173489,-0.012052,-0.084163
2,2002-07-11 13:30:00,0.308214,0.327679,0.303036,0.326786,373676800,2002-07-11,2002-07-11 13:30:00,-1.118450,0.055039,-0.152264
3,2002-07-12 13:30:00,0.331250,0.335536,0.308214,0.312679,443492000,2002-07-12,2002-07-12 13:30:00,-1.162578,-0.044128,-0.122826
4,2002-07-15 13:30:00,0.311250,0.332143,0.300179,0.325536,295993600,2002-07-15,2002-07-15 13:30:00,-1.122282,0.040296,-0.181337
...,...,...,...,...,...,...,...,...,...,...,...
5031,2022-07-01 13:30:00,136.039993,139.039993,135.660004,138.929993,71007500,2022-07-01,2022-07-01 13:30:00,4.933970,0.016035,0.049998
5032,2022-07-05 13:30:00,137.770004,141.610001,136.929993,141.559998,73353800,2022-07-05,2022-07-05 13:30:00,4.952724,0.018753,0.034716
5033,2022-07-06 13:30:00,141.350006,144.119995,141.080002,142.919998,74064300,2022-07-06,2022-07-06 13:30:00,4.962285,0.009561,0.027949
5034,2022-07-07 13:30:00,143.289993,146.550003,143.279999,146.350006,66253700,2022-07-07,2022-07-07 13:30:00,4.986001,0.023716,0.004704


In [6]:

df = pd.read_json('../data/stocks/AAPL/daily/data')

In [7]:
df.tail()

,timestamp,open,high,low,close,volumn,date
5031,2022-07-01 13:30:00,136.039993,139.039993,135.660004,138.929993,71007500,20220701
5032,2022-07-05 13:30:00,137.770004,141.610001,136.929993,141.559998,73353800,20220705
5033,2022-07-06 13:30:00,141.350006,144.119995,141.080002,142.919998,74064300,20220706
5034,2022-07-07 13:30:00,143.289993,146.550003,143.279999,146.350006,66253700,20220707
5035,2022-07-08 13:30:00,145.259995,147.550003,145.000000,147.039993,64493200,20220708


In [8]:
df['date']=pd.to_datetime(df['timestamp']).apply(lambda x: x.date())
df['time']=pd.to_datetime(df['timestamp'])

In [9]:
df.tail()

,timestamp,open,high,low,close,volumn,date,time
5031,2022-07-01 13:30:00,136.039993,139.039993,135.660004,138.929993,71007500,2022-07-01,2022-07-01 13:30:00
5032,2022-07-05 13:30:00,137.770004,141.610001,136.929993,141.559998,73353800,2022-07-05,2022-07-05 13:30:00
5033,2022-07-06 13:30:00,141.350006,144.119995,141.080002,142.919998,74064300,2022-07-06,2022-07-06 13:30:00
5034,2022-07-07 13:30:00,143.289993,146.550003,143.279999,146.350006,66253700,2022-07-07,2022-07-07 13:30:00
5035,2022-07-08 13:30:00,145.259995,147.550003,145.000000,147.039993,64493200,2022-07-08,2022-07-08 13:30:00


In [10]:
df['close_log'] = np.log(df['close'])
df['close_log_diff']=df['close_log'].diff().fillna(0)

In [11]:
df.head()

,timestamp,open,high,low,close,volumn,date,time,close_log,close_log_diff
0,2002-07-09 13:30:00,0.323036,0.326607,0.311786,0.313036,226749600,2002-07-09,2002-07-09 13:30:00,-1.161437,0.000000
1,2002-07-10 13:30:00,0.316250,0.324464,0.308036,0.309286,206880800,2002-07-10,2002-07-10 13:30:00,-1.173489,-0.012052
2,2002-07-11 13:30:00,0.308214,0.327679,0.303036,0.326786,373676800,2002-07-11,2002-07-11 13:30:00,-1.118450,0.055039
3,2002-07-12 13:30:00,0.331250,0.335536,0.308214,0.312679,443492000,2002-07-12,2002-07-12 13:30:00,-1.162578,-0.044128
4,2002-07-15 13:30:00,0.311250,0.332143,0.300179,0.325536,295993600,2002-07-15,2002-07-15 13:30:00,-1.122282,0.040296


In [12]:
df['value']=0
for i in range(20):
    df['value']=df['close_log_diff'].shift(-1).fillna(0)+0.9*df['value'].shift(-1).fillna(0)

In [83]:
df2 = pd.read_json('../data/stocks/AAPL/updown')
df2['toGradeNum']=df2['toGrade'].apply(gradeToNum)


In [84]:
df2['date']=pd.to_datetime(df2['epochGradeDate'].astype('datetime64[s]')).apply(lambda x: x.date())
df2['time']=pd.to_datetime(df2['epochGradeDate'].astype('datetime64[s]'))

In [85]:
df2.tail()

,epochGradeDate,firm,toGrade,fromGrade,action,toGradeNum,date,time
837,1331713260,Canaccord Genuity,Buy,,main,1,2012-03-14,2012-03-14 08:21:00
838,1331705580,Morgan Stanley,Overweight,,main,2,2012-03-14,2012-03-14 06:13:00
839,1331618880,Jefferies,Buy,,main,1,2012-03-13,2012-03-13 06:08:00
840,1331191980,FBN Securities,Outperform,,main,2,2012-03-08,2012-03-08 07:33:00
841,1328775420,Canaccord Genuity,Buy,,main,1,2012-02-09,2012-02-09 08:17:00


In [86]:
wide_table = df2.pivot(index='time', columns='firm', values='toGradeNum').fillna(method='ffill').fillna(3)

In [87]:
occurr = df2.groupby('firm').size().reset_index().sort_values(0, ascending=False)

In [88]:
remove_firm_list = occurr[occurr[0] < 10]['firm']

In [89]:
wide_table.shape

(842, 95)

In [90]:
wide_table.drop(remove_firm_list, axis=1, inplace=True)

In [91]:
wide_table.shape

(842, 28)

In [92]:
remove_firm_list

58        Monness Crespi Hardt
29                 DA Davidson
33          Evercore ISI Group
82                 Sterne Agee
42              Hilliard Lyons
                ...           
24          Cleveland Research
66                  OTR Global
51                     Keybanc
37    Global Equities Research
52                      Lazard
Name: firm, Length: 67, dtype: object

In [93]:
df_merged = pd.merge_asof(df, wide_table.sort_values(by=['time']), on='time')

In [94]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5036 entries, 0 to 5035
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          5036 non-null   datetime64[ns]
 1   open               5036 non-null   float64       
 2   high               5036 non-null   float64       
 3   low                5036 non-null   float64       
 4   close              5036 non-null   float64       
 5   volumn             5036 non-null   int64         
 6   date               5036 non-null   object        
 7   time               5036 non-null   datetime64[ns]
 8   close_log          5036 non-null   float64       
 9   close_log_diff     5036 non-null   float64       
 10  value              5036 non-null   float64       
 11  B of A Securities  2620 non-null   float64       
 12  BMO Capital        2620 non-null   float64       
 13  Baird              2620 non-null   float64       
 14  Bank of 

In [95]:
df_merged = df_merged[df_merged['Baird'].notnull()]

In [96]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2620 entries, 2416 to 5035
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   timestamp          2620 non-null   datetime64[ns]
 1   open               2620 non-null   float64       
 2   high               2620 non-null   float64       
 3   low                2620 non-null   float64       
 4   close              2620 non-null   float64       
 5   volumn             2620 non-null   int64         
 6   date               2620 non-null   object        
 7   time               2620 non-null   datetime64[ns]
 8   close_log          2620 non-null   float64       
 9   close_log_diff     2620 non-null   float64       
 10  value              2620 non-null   float64       
 11  B of A Securities  2620 non-null   float64       
 12  BMO Capital        2620 non-null   float64       
 13  Baird              2620 non-null   float64       
 14  Bank 

In [97]:
values = df_merged.iloc[:, 10:].cov()

In [98]:
df_merged.iloc[:, 10:]

,value,B of A Securities,BMO Capital,Baird,Bank of America,Barclays,Canaccord Genuity,Cascend,Citigroup,Cowen & Co.,...,Oppenheimer,Pacific Crest,PiperJaffray,RBC Capital,Raymond James,Societe Generale,Stifel Nicolaus,UBS,Wedbush,Wells Fargo
2416,0.047130,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2417,0.053485,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2418,0.042829,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2419,0.037539,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
2420,0.066801,3.0,3.0,3.0,3.0,3.0,1.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5031,0.049998,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,...,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
5032,0.034716,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,...,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
5033,0.027949,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,...,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0
5034,0.004704,1.0,3.0,2.0,3.0,3.0,1.0,1.0,1.0,2.0,...,2.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0


In [123]:
df_corr = pd.DataFrame(df_merged.iloc[:, 10:].corr()['value'].sort_values(ascending=False)[1:].reset_index())
df_corr.rename({'index': 'firm', 'value': 'corr'}, axis=1, inplace=True)
df_corr = pd.merge(df_corr, occurr, on = 'firm')

In [124]:
df_corr

,firm,corr,0
0,Oppenheimer,0.144786,13
1,UBS,0.137128,36
2,Barclays,0.135910,43
3,Morgan Stanley,0.134108,61
4,Wedbush,0.133664,26
5,PiperJaffray,0.133490,12
6,Credit Suisse,0.130659,28
7,FBN Securities,0.113160,10
8,JP Morgan,0.111204,22
9,BMO Capital,0.091797,26


In [129]:
wide_table['Canaccord Genuity'].to_csv('Canaccord.csv')
wide_table['Morgan Stanley'].to_csv('MorganStanley.csv')

In [117]:
df_merged.iloc[:, 10:].corr()['value'].sort_values(ascending=False)[1:].reset_index()

,index,value
0,Oppenheimer,0.144786
1,UBS,0.137128
2,Barclays,0.135910
3,Morgan Stanley,0.134108
4,Wedbush,0.133664
5,PiperJaffray,0.133490
6,Credit Suisse,0.130659
7,FBN Securities,0.113160
8,JP Morgan,0.111204
9,BMO Capital,0.091797


In [116]:
df_corr

,firm,corr
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [110]:
df_merged.iloc[:, 10:].corr()['value'].sort_values(ascending=False)[1:]

Oppenheimer          0.144786
UBS                  0.137128
Barclays             0.135910
Morgan Stanley       0.134108
Wedbush              0.133664
PiperJaffray         0.133490
Credit Suisse        0.130659
FBN Securities       0.113160
JP Morgan            0.111204
BMO Capital          0.091797
Societe Generale     0.076933
Stifel Nicolaus      0.052721
Goldman Sachs        0.041196
Citigroup            0.020200
B of A Securities    0.005313
Jefferies           -0.001609
Nomura              -0.007222
Needham             -0.009978
Deutsche Bank       -0.011753
Wells Fargo         -0.030160
Bank of America     -0.049526
Cowen & Co.         -0.058151
Pacific Crest       -0.073381
Baird               -0.073927
Cascend             -0.078561
RBC Capital         -0.089851
Raymond James       -0.096032
Canaccord Genuity   -0.138179
Name: value, dtype: float64

In [105]:
occurr[occurr['firm'] == 'Canaccord Genuity']

,firm,0
18,Canaccord Genuity,41


In [82]:
values['value'].sort_values()

Cascend         -0.003929
DA Davidson     -0.003065
Mizuho          -0.002758
Argus           -0.002669
Raymond James   -0.002636
                   ...   
Goldman Sachs    0.002972
Barclays         0.003131
Credit Suisse    0.003267
HSBC             0.003552
Rosenblatt       0.006811
Name: value, Length: 96, dtype: float64

In [1]:
values['value'].sort_values(ascending=False)

NameError: name 'values' is not defined